In [ ]:
# set up Dependencies
import pandas as pd
import numpy as np
import requests
import time
import os
import matplotlib.pyplot as plt
from scipy.stats import linregress
from citipy import citipy

weather_api_key = os.environ.get('WEATHER_API_KEY')
